## Fine-Tune InceptionV3 on CIFAR10

- CIFAR10 is a dataset with **10 classes**
- InceptionV3 is a pre-trained model by ImageNet with **1000 classes**

In order to output reliable prediction sets, we need to fine-tune the full-connected layer of model

# 1. Load Model

Generally, a model has two mode: train mode and evaluation mode. 

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models

# check GPU status
print("Is CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

# load pre-trained model InceptionV3 and set mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)

Is CUDA available: True
Device count: 1
Device name: NVIDIA GeForce RTX 3060 Ti


C:\Users\jiayang\anaconda3\envs\pytorch2\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\jiayang\anaconda3\envs\pytorch2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# 1. Fine-Tune prepare -- load train data set

Load a CIFAR10 as the train data to fine-tune Inception

In [13]:
import torchvision.transforms as transforms               # include image preprocess tools
from torchvision.datasets import CIFAR10, CIFAR100        # for loading images from Pytorch CIFAR
from torch.utils.data import DataLoader
import os

# reprocess the images from CIFAR
train_data_transform = transforms.Compose([ 
    transforms.RandomCrop(32, padding=4),  # 随机裁剪
    transforms.RandomHorizontalFlip(),    
    transforms.ToTensor(),          
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])  # normalize
])

data_transform = transforms.Compose([ 
    transforms.ToTensor(),          # transfer to tensor
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])  # normalize
])

# make sure CIFAR10 and CIFAR100 in the following adress:  THE_ADRESS_IN_OUTPUT/data
print("Current Working Directory:", os.getcwd())

# load data set for training
pre_train_dataset = CIFAR10(root="./data", train=True, download=True,transform=train_data_transform)
pre_test_dataset = CIFAR10(root="./data", train=True, download=True, transform=data_transform)

pre_train_loader = DataLoader(pre_train_dataset, batch_size=32, shuffle=True)
pre_test_loader = DataLoader(pre_test_dataset, batch_size=32, shuffle=False)

Current Working Directory: C:\Users\jiayang\ipynb
Files already downloaded and verified
Files already downloaded and verified


# 2.1 Fine-Tune

- adjust output dimension from 1000 to 10
- Freeze parameters in convolution layers and unlock parameters in fc layers
- Train fc layer with CIFAR10

In [15]:
import torch.optim as optim # optimizer

# adjust output dimension 1000 --> 10
model.fc = nn.Linear(model.fc.in_features, 10)

model = model.to(device)
        
# Train fc layer
loss_function = nn.CrossEntropyLoss()   # define loss function
optimizer = optim.Adam(model.fc.parameters(), lr=0.01)  # define optimize for fc layer

epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0 # total loss in this epoch
    for images, labels in pre_train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # front propagation
        outputs = model(images)
        loss = loss_function(outputs, labels)

        # back propagation
        optimizer.zero_grad()  # clear gradient
        loss.backward()        # optimize parameters by back propagation
        optimizer.step()       # update the parameters

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}")

# Test model after training
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in pre_test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        #  calculate the correct rate after training
        _, predicted = torch.max(outputs, 1)  # outputs: [batch_size, num_classes]  --torch.max--> max_predicted_prob, max_predicted_prob_label
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Epoch [1/10], Loss: 4493.9840
Epoch [2/10], Loss: 4479.0306


KeyboardInterrupt: 

In [12]:
epochs = 10
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
for epoch in range(epochs):
    model.train()
    total_loss = 0 # total loss in this epoch
    for images, labels in pre_train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # front propagation
        outputs = model(images)
        loss = loss_function(outputs, labels)

        # back propagation
        optimizer.zero_grad()  # clear gradient
        loss.backward()        # optimize parameters by back propagation
        optimizer.step()       # update the parameters

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}")

# Test model after training
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in pre_test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        #  calculate the correct rate after training
        _, predicted = torch.max(outputs, 1)  # outputs: [batch_size, num_classes]  --torch.max--> max_predicted_prob, max_predicted_prob_label
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Epoch [1/10], Loss: 2007.7259
Epoch [2/10], Loss: 1995.7321
Epoch [3/10], Loss: 1991.3193
Epoch [4/10], Loss: 1983.5468
Epoch [5/10], Loss: 1966.1875
Epoch [6/10], Loss: 1980.0112
Epoch [7/10], Loss: 1949.0931
Epoch [8/10], Loss: 1952.4309
Epoch [9/10], Loss: 1960.0724
Epoch [10/10], Loss: 1944.1561
Test Accuracy: 59.83%


# 1.5 Save fine-tuned Model

Save current parameters and load the trained InceptionV3 in future by following steps:
- model = models.inception_v3(pretrained=False)  # Noticed: pretrained=False!!!
- model.fc = nn.Linear(model.fc.in_features, 10)
- model_path = "C:\Users\jiayang\ipynb\trainedModel\ResNet_50_CIFAR10.pth"
- model.load_state_dict(torch.load(model_path))
- model.eval()
- print(f"Model loaded from {model_path}")

In [6]:
images, labels = next(iter(pre_test_loader))
images = images.to(device)

outputs = model(images)
print("Model Output Shape:", outputs.shape)  # [batch_size, 10]

# create directory
save_dir = "C:\\Users\\jiayang\\ipynb\\trainedModel" # your save save path
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# save model data
model_path = os.path.join(save_dir, "ResNet_50_CIFAR10.pth")
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

Model Output Shape: torch.Size([32, 10])
Model saved to C:\Users\jiayang\ipynb\trainedModel\ResNet_50_CIFAR10.pth


In [7]:
import numpy as np
 
model = model.to(device)
model.eval()
print(f"Model loaded successfully from {model_path}")

dataset = CIFAR10(root="./data", train=False, download=True, transform=data_transform)

# randomly select 25 pictures
subset_indices = np.random.choice(len(dataset), 25, replace=False)
subset = torch.utils.data.Subset(dataset, subset_indices)
loader = torch.utils.data.DataLoader(subset, batch_size=1, shuffle=False)

# CIFAR-10 labels
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

print("\nPredictions on 25 images:")
for i, (image, true_label) in enumerate(loader):
    image, true_label = image.to(device), true_label.item()
    
    outputs = model(image)
    softmax_probs = torch.softmax(outputs, dim=1).squeeze(0) 
    
    print(f"Image {i+1}:")
    print(f"  Correct Label Index: {true_label} ({classes[true_label]})")
    print(f"  Softmax Probabilities: {softmax_probs.tolist()}") 
    print()

Model loaded successfully from C:\Users\jiayang\ipynb\trainedModel\ResNet_50_CIFAR10.pth
Files already downloaded and verified

Predictions on 25 images:
Image 1:
  Correct Label Index: 9 (truck)
  Softmax Probabilities: [5.310959295456996e-06, 0.0049262274987995625, 7.131742313504219e-05, 0.002111848909407854, 0.0008120554266497493, 0.0012696977937594056, 1.0165010735363467e-06, 0.0001365264761261642, 1.342230007139733e-05, 0.9906526803970337]

Image 2:
  Correct Label Index: 6 (frog)
  Softmax Probabilities: [8.79492836247664e-06, 1.0678945727704559e-06, 0.04288917034864426, 0.5719044208526611, 0.00030687625985592604, 0.0008152229129336774, 0.3839711844921112, 4.591902325046249e-05, 4.52903586847242e-05, 1.1959174116782378e-05]

Image 3:
  Correct Label Index: 5 (dog)
  Softmax Probabilities: [0.0014166398905217648, 9.126879012910649e-05, 0.0005561573198065162, 0.0017897114157676697, 0.0008043101406656206, 0.9540363550186157, 3.3530730433994904e-05, 0.0008052687626332045, 0.039724461